In [1]:
from utils import extract_data, load_data
import os
import pickle
import pandas as pd
from IPython.display import display

In [2]:
def extract(model: str):
    raw_data = load_data(os.path.join('data', 'raw', f'bonbanh.{model}.pkl'))
    clean_data = []
    for e in raw_data:
        clean_data.append(extract_data(e))
    df = pd.DataFrame(clean_data)
    display(df.head())
    df.to_pickle(os.path.join('data', 'clean', f'bonbanh.{model}.pkl'))

In [3]:
# extract('audi')
# extract('bentley')
# extract('bmw')
# extract('chevrolet')
# extract('daewoo')
# extract('ford')
# extract('honda')
# extract('hyundai')
# extract('isuzu')
# extract('jeep')
# extract('kia')
# extract('landrover')
# extract('lexus')
# extract('mazda')
# extract('mercedes_benz')
# extract('mg')
# extract('mini')
# extract('mitsubishi')
# extract('nissan')
# extract('peugeot')
# extract('porsche')
# extract('subaru')
# extract('suzuki')
# extract('toyota')
# extract('vinfast')
# extract('volkswagen')
# extract('volvo')

In [4]:
def merge_data(folder_path):
    data_df = pd.DataFrame()
    files = os.listdir(folder_path)
    sorted_files = sorted(files)

    for file_name in sorted_files:
        if file_name.startswith('bonbanh'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'rb') as f:
                content = pickle.load(f)
                if isinstance(content, pd.DataFrame):
                    data_df = pd.concat([data_df, content], ignore_index=True)
                    print(f"{file_name}: {content.shape[0]}")

    data_df.drop(columns=["Mô tả"], axis=1, inplace=True)
    return data_df

In [5]:
FOLDER_PATH = os.path.join("data", "clean")
cars = merge_data(FOLDER_PATH)

bonbanh.audi.pkl: 315
bonbanh.bentley.pkl: 65
bonbanh.bmw.pkl: 682
bonbanh.chevrolet.pkl: 554
bonbanh.daewoo.pkl: 248
bonbanh.ford.pkl: 3155
bonbanh.honda.pkl: 1352
bonbanh.hyundai.pkl: 3475
bonbanh.isuzu.pkl: 120
bonbanh.jeep.pkl: 89
bonbanh.kia.pkl: 2900
bonbanh.landrover.pkl: 411
bonbanh.lexus.pkl: 1177
bonbanh.mazda.pkl: 2147
bonbanh.mercedes_benz.pkl: 3329
bonbanh.mg.pkl: 376
bonbanh.mini.pkl: 113
bonbanh.mitsubishi.pkl: 1474
bonbanh.nissan.pkl: 300
bonbanh.peugeot.pkl: 275
bonbanh.porsche.pkl: 429
bonbanh.subaru.pkl: 212
bonbanh.suzuki.pkl: 343
bonbanh.toyota.pkl: 5860
bonbanh.vinfast.pkl: 969
bonbanh.volkswagen.pkl: 361
bonbanh.volvo.pkl: 150


# Clean

Loading the dataset

In [6]:
cars.head()

,URL,Tình trạng chung,Tên xe,Giá,Tỉnh,Mã xe,Năm sản xuất,Tình trạng,Số Km đã đi,Xuất xứ,Kiểu dáng,Hộp số,Động cơ,Màu ngoại thất,Màu nội thất,Số chỗ ngồi,Số cửa,Dẫn động
0,https://bonbanh.com/xe-audi-a8-l-55-tfsi-quatt...,Xe cũ,Audi A8 L 55 TFSI Quattro - 2020,2950000000,TP HCM,5596529,2020,Xe đã dùng,"30,000 Km",Nhập khẩu,Sedan,Số tự động,Xăng 3.0 L,Đen,Đen,5 chỗ,4 cửa,AWD - 4 bánh toàn thời gian
1,https://bonbanh.com/xe-audi-a6-2.0-tfsi-2014-5...,Xe cũ,Audi A6 2.0 TFSI - 2014,560000000,Hà Nội,5555194,2014,Xe đã dùng,"94,000 Km",Nhập khẩu,Sedan,Số tự động,Xăng 2.0 L,Xanh,Nâu,5 chỗ,4 cửa,FWD - Dẫn động cầu trước
2,https://bonbanh.com/xe-audi-a4-2.0-tfsi-2016-5...,Xe cũ,Audi A4 2.0 TFSI - 2016,690000000,TP HCM,5464113,2016,Xe đã dùng,"50,000 Km",Nhập khẩu,Sedan,Số tự động,Xăng 2.0 L,Đen,Đen,5 chỗ,4 cửa,FWD - Dẫn động cầu trước
3,https://bonbanh.com/xe-audi-q3-2.0-quattro-201...,Xe cũ,Audi Q3 2.0 Quattro - 2016,699000000,Hà Nội,5532313,2016,Xe đã dùng,"76,000 Km",Nhập khẩu,Crossover,Số tự động,Xăng 2.0 L,Xanh,Đen,5 chỗ,5 cửa,AWD - 4 bánh toàn thời gian
4,https://bonbanh.com/xe-audi-a4-2.0-tfsi-2017-5...,Xe cũ,Audi A4 2.0 TFSI - 2017,799000000,TP HCM,5346160,2017,Xe đã dùng,0 Km,Nhập khẩu,Sedan,Số tự động,Xăng 2.0 L,Đen,Nâu,5 chỗ,4 cửa,FWD - Dẫn động cầu trước


In [7]:
# Mô tả các cột
cars.describe()

,URL,Tình trạng chung,Tên xe,Giá,Tỉnh,Mã xe,Năm sản xuất,Tình trạng,Số Km đã đi,Xuất xứ,Kiểu dáng,Hộp số,Động cơ,Màu ngoại thất,Màu nội thất,Số chỗ ngồi,Số cửa,Dẫn động
count,30881,30881,30881,30881,30881,30881,30881,30881,24600,30881,30881,30881,30881,30881,30881,30881,30881,30881
unique,30514,2,4743,2194,63,30514,36,2,1780,2,10,2,89,18,18,21,7,5
top,https://bonbanh.com/xe-ford-ranger-xls-2.0l-4x...,Xe cũ,Toyota Land Cruiser 3.5 V6 - 2024,495000000,Hà Nội,5592465,2024,Xe đã dùng,0 Km,Lắp ráp trong nước,SUV,Số tự động,Xăng 2.0 L,Trắng,Đen,5 chỗ,5 cửa,FWD - Dẫn động cầu trước
freq,2,24600,144,192,16834,2,4603,24600,3864,17464,12041,26521,7413,10518,13781,20823,18314,16488


In [8]:
# Thông tin của dataframe
cars.info()

# Xóa các hàng trùng lặp
cars = cars.drop_duplicates(subset=['Mã xe'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30881 entries, 0 to 30880
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   URL               30881 non-null  object
 1   Tình trạng chung  30881 non-null  object
 2   Tên xe            30881 non-null  object
 3   Giá               30881 non-null  object
 4   Tỉnh              30881 non-null  object
 5   Mã xe             30881 non-null  object
 6   Năm sản xuất      30881 non-null  object
 7   Tình trạng        30881 non-null  object
 8   Số Km đã đi       24600 non-null  object
 9   Xuất xứ           30881 non-null  object
 10  Kiểu dáng         30881 non-null  object
 11  Hộp số            30881 non-null  object
 12  Động cơ           30881 non-null  object
 13  Màu ngoại thất    30881 non-null  object
 14  Màu nội thất      30881 non-null  object
 15  Số chỗ ngồi       30881 non-null  object
 16  Số cửa            30881 non-null  object
 17  Dẫn động    

In [9]:
# Các giá trị của mỗi cột
cars.nunique()

URL                 30514
Tình trạng chung        2
Tên xe               4743
Giá                  2194
Tỉnh                   63
Mã xe               30514
Năm sản xuất           36
Tình trạng              2
Số Km đã đi          1780
Xuất xứ                 2
Kiểu dáng              10
Hộp số                  2
Động cơ                89
Màu ngoại thất         18
Màu nội thất           18
Số chỗ ngồi            21
Số cửa                  7
Dẫn động                5
dtype: int64

In [10]:
# Các giá trị còn thiếu
cars.isnull().sum()

URL                    0
Tình trạng chung       0
Tên xe                 0
Giá                    0
Tỉnh                   0
Mã xe                  0
Năm sản xuất           0
Tình trạng             0
Số Km đã đi         6204
Xuất xứ                0
Kiểu dáng              0
Hộp số                 0
Động cơ                0
Màu ngoại thất         0
Màu nội thất           0
Số chỗ ngồi            0
Số cửa                 0
Dẫn động               0
dtype: int64

In [11]:
# Loại bỏ phần ' km' và chuyển đổi kiểu dữ liệu của cột 'Số Km đã đi'
cars['Số Km đã đi'] = cars['Số Km đã đi'].str.replace(' Km', '').str.replace(',', '')
cars['Số Km đã đi'] = cars['Số Km đã đi'].fillna(0)
cars['Số Km đã đi'] = cars['Số Km đã đi'].astype(int)

In [12]:
# Gán giá trị 0 cho cột 'Số Km đã đi' nếu cột 'Tình trạng' là 'Xe mới'
cars.loc[cars['Tình trạng'] == 'Xe mới', 'Số Km đã đi'] = 0

In [13]:
cars.isnull().sum()

URL                 0
Tình trạng chung    0
Tên xe              0
Giá                 0
Tỉnh                0
Mã xe               0
Năm sản xuất        0
Tình trạng          0
Số Km đã đi         0
Xuất xứ             0
Kiểu dáng           0
Hộp số              0
Động cơ             0
Màu ngoại thất      0
Màu nội thất        0
Số chỗ ngồi         0
Số cửa              0
Dẫn động            0
dtype: int64

In [14]:
# Xóa những hàng sai ở cột "Số km đã đi"
# Lọc ra những hàng cần xóa
rows_to_drop = cars[(cars['Tình trạng'] == 'Xe đã dùng') & (cars['Số Km đã đi'] == 0)].index

# Xóa các hàng tương ứng
cars = cars.drop(rows_to_drop)

In [15]:
cars["Động cơ"].unique()

array(['Xăng  3.0 L', 'Xăng  2.0 L', 'Xăng  4.2 L', 'Xăng  4.0 L',
       'Xăng  3.6 L', 'Xăng  1.4 L', 'Xăng  3.2 L', 'Xăng  1.8 L', 'Điện',
       'Xăng  5.2 L', 'Xăng', 'Xăng  1.0 L', 'Xăng  6.0 L', 'Xăng  6.8 L',
       'Xăng  1.5 L', 'Hybrid  4.4 L', 'Xăng  1.6 L', 'Xăng  4.8 L',
       'Xăng  4.4 L', 'Hybrid  1.5 L', 'Xăng  2.5 L', 'Xăng  5.0 L',
       'Dầu  3.0 L', 'Dầu  2.5 L', 'Xăng  1.2 L', 'Xăng  2.4 L',
       'Dầu  2.8 L', 'Xăng  0.8 L', 'Dầu  2.0 L', 'Dầu  2.4 L',
       'Xăng  2.3 L', 'Dầu  2.2 L', 'Xăng  3.5 L', 'Dầu  3.2 L', 'Dầu',
       'Dầu  6.7 L', 'Xăng  2.6 L', 'Xăng  4.6 L', 'Hybrid  2.0 L',
       'Xăng  2.2 L', 'Xăng  1.25 L', 'Xăng  2.7 L', 'Hybrid  1.6 L',
       'Xăng  1.1 L', 'Dầu  3.9 L', 'Xăng  3.8 L', 'Dầu  11.1 L',
       'Xăng  3.3 L', 'Dầu  5.2 L', 'Dầu  1.9 L', 'Dầu  3.6 L',
       'Dầu  1.7 L', 'Dầu  4.4 L', 'Hybrid  3.0 L', 'Xăng  3.4 L',
       'Xăng  4.7 L', 'Xăng  5.7 L', 'Hybrid  2.4 L', 'Hybrid  2.5 L',
       'Hybrid  3.5 L', 'Xăng  4.3 L',

In [16]:
# Tách loại động cơ từ cột động cơ
cars['Loại động cơ'] = cars['Động cơ'].apply(lambda x: x.split(' ')[0])
# Tách dung tích từ cột động cơ
cars['Dung tích'] = cars['Động cơ'].apply(lambda x: float(x.split(' ')[-2]) if len(x.split(' ')) > 1 else None)

In [17]:
nan_count = cars["Dung tích"].isna().sum()
print("Số lượng giá trị NaN trong cột 'Dung tích':", nan_count)

Số lượng giá trị NaN trong cột 'Dung tích': 278


In [18]:
cars.dropna(subset=["Dung tích"], inplace=True)

In [19]:
def cut_year(name):
    if "-" in name:
        name = name.rsplit("-", 1)[0]
    return name.strip()

cars["Tên xe"] = cars["Tên xe"].apply(cut_year)
cars["Tên hãng xe"] = cars["Tên xe"].apply(
    lambda x: " ".join(x.split(" ")[:2]) if "Mercedes" in x else x.split(" ")[0]
)
cars["Tên dòng xe"] = cars["Tên xe"].apply(
    lambda x: " ".join(x.split(" ")[2:]) if "Mercedes" in x else " ".join(x.split(" ")[1:])
)

In [20]:
# Xóa các cột 'URL' 'Mã xe', 'Tình trạng chung', 'Mô tả'
cars = cars.drop(["Tên xe", "Tình trạng chung", "Động cơ"], axis=1)

# Định nghĩa thứ tự mới của các cột
new_order = [
    "Mã xe", "URL", "Tên hãng xe", "Tên dòng xe", "Năm sản xuất", 
    "Tỉnh", "Tình trạng", "Số Km đã đi",  "Xuất xứ", "Kiểu dáng", 
    "Hộp số", "Màu ngoại thất", "Màu nội thất", "Số chỗ ngồi",
    "Số cửa", "Dẫn động", "Loại động cơ", "Dung tích", "Giá",
]

# Sắp xếp lại các cột theo thứ tự mới
cars = cars.reindex(columns=new_order)

In [21]:
# Đổi kiểu dữ liệu năm
cars['Năm sản xuất'] = cars['Năm sản xuất'].astype(int)

# Chuyển đổi kiểu dữ liệu của cột 'Giá xe' thành float
cars['Giá'] = cars['Giá'].astype(float) / 1000000
cars['Giá'] = cars['Giá'].astype(float)

# Chuyển đổi cột 'số chỗ ngồi' thành int
cars['Số chỗ ngồi'] = cars['Số chỗ ngồi'].str.split(' ').str[0].astype(int)

# Chuyển đổi cột 'số cửa' thành int
cars['Số cửa'] = cars['Số cửa'].str.split(' ').str[0].astype(int)

# Dẫn động
cars = cars[cars["Dẫn động"] != "-"]
cars["Dẫn động"] = cars["Dẫn động"].apply(lambda x: str(x).split()[0])

# Rút gọn tình trạng
cars["Tình trạng"] = cars["Tình trạng"].replace({"Xe đã dùng": "Cũ", "Xe mới": "Mới"})

# Reset index
cars.reset_index(drop=True, inplace=True)

In [22]:
print(cars.shape)
cars.head()

(26406, 19)


,Mã xe,URL,Tên hãng xe,Tên dòng xe,Năm sản xuất,Tỉnh,Tình trạng,Số Km đã đi,Xuất xứ,Kiểu dáng,Hộp số,Màu ngoại thất,Màu nội thất,Số chỗ ngồi,Số cửa,Dẫn động,Loại động cơ,Dung tích,Giá
0,5596529,https://bonbanh.com/xe-audi-a8-l-55-tfsi-quatt...,Audi,A8 L 55 TFSI Quattro,2020,TP HCM,Cũ,30000,Nhập khẩu,Sedan,Số tự động,Đen,Đen,5,4,AWD,Xăng,3.0,2950.0
1,5555194,https://bonbanh.com/xe-audi-a6-2.0-tfsi-2014-5...,Audi,A6 2.0 TFSI,2014,Hà Nội,Cũ,94000,Nhập khẩu,Sedan,Số tự động,Xanh,Nâu,5,4,FWD,Xăng,2.0,560.0
2,5464113,https://bonbanh.com/xe-audi-a4-2.0-tfsi-2016-5...,Audi,A4 2.0 TFSI,2016,TP HCM,Cũ,50000,Nhập khẩu,Sedan,Số tự động,Đen,Đen,5,4,FWD,Xăng,2.0,690.0
3,5532313,https://bonbanh.com/xe-audi-q3-2.0-quattro-201...,Audi,Q3 2.0 Quattro,2016,Hà Nội,Cũ,76000,Nhập khẩu,Crossover,Số tự động,Xanh,Đen,5,5,AWD,Xăng,2.0,699.0
4,5524426,https://bonbanh.com/xe-audi-q5-2.0-at-2016-552...,Audi,Q5 2.0 AT,2016,Hà Nội,Cũ,90000,Nhập khẩu,SUV,Số tự động,Trắng,Kem,5,5,AWD,Xăng,2.0,739.0


In [23]:
FOLDER_PATH = os.path.join("data", "clean")

# Lưu dataframe vào file pickle
cars.to_pickle(os.path.join(FOLDER_PATH, 'cars.pkl')) 

# Lưu dataframe vào file Excel
cars.to_excel(os.path.join(FOLDER_PATH, 'cars.xlsx'), index=False)

# Lưu dataframe vào file csv
cars.to_csv(os.path.join(FOLDER_PATH, "cars.csv"), index=False)